In [4]:
import h2o
h2o.init()
df = h2o.import_file('../data/train.csv')



No instance found at ip and port: localhost:54321. Trying to start local jar...


JVM stdout: /tmp/tmpylughc/h2o_saket_started_from_python.out
JVM stderr: /tmp/tmpgSMxVd/h2o_saket_started_from_python.err
Using ice_root: /tmp/tmpH1an7A


Java Version: java version "1.7.0_85"
OpenJDK Runtime Environment (IcedTea 2.6.1) (7u85-2.6.1-5ubuntu0.14.04.1)
OpenJDK 64-Bit Server VM (build 24.85-b03, mixed mode)


Starting H2O JVM and connecting: ....................... Connection sucessful!


H2O cluster uptime:,2 seconds 163 milliseconds
H2O cluster version:,3.2.0.9
H2O cluster name:,H2O_started_from_python
H2O cluster total nodes:,1
H2O cluster total memory:,3.46 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321



Parse Progress: [##################################################] 100%
Imported ../data/train.csv. Parsed 13,765,201 rows and 24 cols


In [6]:
import pandas as pd


In [7]:
def marshall_palmer(ref, minutes_past):
    #minutes_past = df['minutes_past']

    minutes_in_segment = 60.0
    valid_time = minutes_past.diff()
    valid_time.iloc[0] = minutes_past.iloc[0]
    valid_time.iloc[-1] = valid_time.iloc[-1] + minutes_in_segment - np.sum(valid_time)
    valid_time = valid_time/60.0
    valid_time = valid_time[np.isfinite(ref)]
    ref = ref[np.isfinite(ref)]
    mmperhr = np.dot(pow(pow(10, ref/10)/200, 0.625), valid_time)
    return mmperhr

def operate_on_each_hour(hour):
    hour = hour.sort('minutes_past', ascending=True)
    gauge_estimate = marshall_palmer(hour['Ref'], hour['minutes_past'])
    #hour['gauge_estimated'] = gauge_estimate
    return gauge_estimate

In [ ]:
traindata = pd.read_csv('../data/train.csv')
traindata = traindata.set_index('Id')
traindata['estimated'] = traindata.groupby(traindata.index).apply(operate_on_each_hour)
traindata['estimated']